---
title: Assignment 04
author:
  - name: Bhargavi Manyala
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
number-sections: true
date: today
date-modified: today
date-format: long
format:
  html:
    theme: cerulean
    toc: true
    toc-depth: 2
engine: jupyter
jupyter: assignment-04-kernel
execute:
  echo: true
  eval: true
  output: true
  freeze: auto
---

# Load the Dataset

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np
from pyspark.sql.functions import col, pow
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, regexp_replace, trim


np.random.seed(42)

pio.renderers.default = "notebook+notebook_connected+vscode"

# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("multiLine", "true")
    .option("escape", "\"")  
    .csv("data/lightcast_job_postings.csv")

)

df.createOrReplaceTempView("job_postings")
#df.show(5)


/home/ubuntu/assignment-04-BhargaviManyala/.venv/lib/python3.12/site-packages/kaleido/__init__.py:14: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/29 19:47:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/29 19:47:58 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


# Feature Engineering

### Selection of Variables and Dropping NA's
* I referred to Claude to decide which cast (Float, Double, or Integer) should be used for numeric columns.

In [4]:

# Variable Selection
min_years = "MIN_YEARS_EXPERIENCE"
cont_cols = ["MAX_YEARS_EXPERIENCE", "DURATION", "SALARY_FROM"]
cat_cols = ["EMPLOYMENT_TYPE_NAME", "EDUCATION_LEVELS_NAME"]
y_col = "SALARY"

# Dropping NA's
required_cols = [min_years] + cont_cols + cat_cols + [y_col]
df_clean = df.dropna(subset=required_cols)

# Clean EDUCATION_LEVELS_NAME
df_clean = df_clean.withColumn(
    "EDUCATION_LEVELS_NAME",
    trim(regexp_replace(col("EDUCATION_LEVELS_NAME"), r"[\[\]\n\"]", ""))
)

 #  Cast numeric columns to double 
for c in [y_col, min_years] + cont_cols:
    df_clean = df_clean.withColumn(c, col(c).cast("double"))

# Data Cleaned
print(" Data cleaned and numeric columns casted ")




 Data cleaned and numeric columns casted 
